## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-25-01-15-18 +0000,nyt,Kash Patel’s Olympics Trip Left Plenty of Time...,https://www.nytimes.com/2026/02/24/us/politics...
1,2026-02-25-01-08-55 +0000,nyt,How The Times Reports on the State of the Union,https://www.nytimes.com/2026/02/24/insider/new...
2,2026-02-25-01-08-50 +0000,nyt,68% of Americans Say Trump Has the Wrong Prior...,https://www.nytimes.com/2026/02/24/us/politics...
3,2026-02-25-01-08-49 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/24/us/tru...
4,2026-02-25-01-08-49 +0000,nyt,Trump to Address a Complacent Congress Badly S...,https://www.nytimes.com/2026/02/24/us/congress...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2435/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,67
12,union,28
175,new,28
34,war,23
778,mexico,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
400,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...,202
319,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...,177
351,2026-02-24-10-30-00 +0000,wsj,Tuesday’s State of the Union address gives Pre...,https://www.wsj.com/economy/sizing-up-the-u-s-...,164
306,2026-02-24-13-00-00 +0000,wsj,Dozens of congressional Democrats are preparin...,https://www.wsj.com/politics/policy/democrats-...,145
234,2026-02-24-16-59-59 +0000,nypost,Trump weighs new batch of national security ta...,https://nypost.com/2026/02/24/business/trump-w...,139


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
400,202,2026-02-24-02-00-00 +0000,wsj,President Trump will use his State of the Unio...,https://www.wsj.com/politics/policy/trump-to-s...
403,72,2026-02-24-01-22-46 +0000,cbc,Will El Mencho's cartel wage 'total war' again...,https://www.cbc.ca/player/play/9.7103450?cmp=rss
32,62,2026-02-24-23-54-43 +0000,nypost,Warner Bros. Discovery may upend Netflix deal ...,https://nypost.com/2026/02/24/media/warner-bro...
319,61,2026-02-24-11-54-00 +0000,wsj,"A bit awkward? Supreme Court justices, some of...",https://www.wsj.com/politics/trump-to-face-off...
156,52,2026-02-24-20-13-35 +0000,bbc,Mandelson's lawyers say he was arrested over '...,https://www.bbc.com/news/articles/c05vey47yddo...
150,48,2026-02-24-20-26-51 +0000,nypost,"Ozempic, Wegovy prices to drop up to 50% as No...",https://nypost.com/2026/02/24/business/ozempic...
120,47,2026-02-24-21-38-25 +0000,nypost,Defense Sec. Pete Hegseth gives Anthropic Frid...,https://nypost.com/2026/02/24/business/defense...
361,46,2026-02-24-09-00-00 +0000,cbc,'Putin will not stop': Zelenskyy warns world o...,https://www.cbc.ca/news/world/zelenskyy-trump-...
381,41,2026-02-24-04-21-44 +0000,nyt,"Peter Mandelson, Ex-Ambassador to the U.S., Is...",https://www.nytimes.com/2026/02/23/world/europ...
340,39,2026-02-24-11-00-00 +0000,latimes,"Chabria: At San Quentin, Newsom shows off the ...",https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
